<a href="https://colab.research.google.com/github/antonio1134/IA-ComputerVision/blob/main/proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Actualizar el sistema e instalar Python 3.8 y su entorno virtual
!sudo apt-get update -y
!sudo apt-get install python3.8 python3.8-dev python3.8-venv -y

# Crear y activar el entorno virtual
!python3.8 -m venv tfjs-venv
!source tfjs-venv/bin/activate

# Actualizar pip e instalar TensorFlow y TensorFlow.js
!tfjs-venv/bin/pip install --upgrade pip
!tfjs-venv/bin/pip install tensorflow==2.8 tensorflowjs tensorflow-decision-forests tensorflow_datasets

# Configurar entorno Keras
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [ ]:
!apt-get install libhdf5-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libhdf5-dev is already the newest version (1.10.7+repack-4ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 58 not upgraded.


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Función para crear carpetas si no existen
def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

# subir carpeta al entorno de colab
print("por favor, sube tu carpeta de imágenes (organizada por clases en subcarpetas):")
from google.colab import files
import zipfile

uploaded = files.upload()  # sube archivo
uploaded_filename = list(uploaded.keys())[0]  # Obtiene el nombre del archivo subido

# Descompre el archivo subido
dataset_path = '/content/dataset'
with zipfile.ZipFile(uploaded_filename, 'r') as zip_ref:
    zip_ref.extractall(dataset_path)
print(f"Carpeta descomprimida en: {dataset_path}")

# Preprocesamiento las imágenes
def preprocess_image(image, target_size=(64, 64)):
    if isinstance(image, str):  # Si es una ruta, carga la imagen
        image = cv2.imread(image)
        if image is None:
            raise FileNotFoundError(f"No se pudo leer la imagen en la ruta: {image}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    return image / 255.0  # Normalizar entre 0 y 1

# Cargar imágenes desde carpetas
def load_images_from_folders(folder_paths, target_size=(64, 64)):
    images = []
    image_labels = []
    class_labels = sorted(os.listdir(folder_paths))  # Nombre carpetas
    label_map = {name: idx for idx, name in enumerate(class_labels)}
    for label_name, label in label_map.items():
        folder = os.path.join(folder_paths, label_name)
        for filename in os.listdir(folder):
            image_path = os.path.join(folder, filename)
            if filename.endswith(('.png', '.jpg', '.jpeg')):  # Extensiones permitidas
                try:
                    img = preprocess_image(image_path, target_size)
                    images.append(img)
                    image_labels.append(label)
                except Exception as e:
                    print(f"Error cargando la imagen {image_path}: {e}")
    return np.array(images), np.array(image_labels), label_map

# Carga imágenes y etiquetas
print("Cargando imágenes desde la carpeta descomprimida...")
X, y, label_map = load_images_from_folders(dataset_path)
num_classes = len(label_map)  # Total de clases
y = to_categorical(y, num_classes=num_classes)  # Codificación one-hot

# Divide los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')  # Clases de frutas y verduras
])

# Compilacion modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento modelo
print("Entrenando el modelo...")
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32)

# Creacion carpeta para modelos
output_path = '/content/models'
create_folder(output_path)

# Guardar el modelo en formato H5
h5_model_path = os.path.join(output_path, 'fruit_veg_classifier.h5')
model.save(h5_model_path)
print(f"Modelo guardado en: {h5_model_path}")

# Guardar como JSON y BIN
output_path = "./models"
os.makedirs(output_path, exist_ok=True)
json_path = os.path.join(output_path, 'fruit_veg_classifier.json')
model_json = model.to_json()
with open(json_path, "w") as json_file:
    json_file.write(model_json)

# Guardar los pesos en un único archivo BIN
bin_path = os.path.join(output_path, 'fruit_veg_classifier.bin')
weights = model.get_weights()
with open(bin_path, "wb") as bin_file:
    for weight in weights:
        np.save(bin_file, weight)
print("Modelo guardado en los formatos H5, JSON y BIN.")

# Función para predecir el tipo
def predict_category(image, model, target_size=(64, 64), label_map=None):
    img = preprocess_image(image, target_size)
    img = np.expand_dims(img, axis=0)  # Añadir dimensión para el batch
    predictions = model.predict(img)
    class_idx = np.argmax(predictions)
    label = [name for name, idx in label_map.items() if idx == class_idx][0]
    return label




In [ ]:
# Ejemplo de predicción
test_image_path = '/content/dataset/Brocoli/brocoli.jpg'  # Sube imagen para probar
if os.path.exists(test_image_path):
    print("Cargando modelo para realizar predicción...")
    model = tf.keras.models.load_model(h5_model_path)
    predicted_label = predict_category(test_image_path, model, label_map=label_map)
    print(f"Prediccion: {predicted_label}")
else:
    print(f"No se encontró la imagen de prueba en: {test_image_path}")

Cargando modelo para realizar predicción...
1/1 [==============================] - 0s 119ms/step
Prediccion: Brocoli
